# Setup
We need datasets, dataloaders, and code to train and evaluate models

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from collections import defaultdict

In [ ]:
class RULDataset(Dataset):
    def __init__(self, X, y, dim="1d"):
        """
        Initialize the dataset by passing a DataFrame.
        Args:
            dataframe (pd.DataFrame): DataFrame with pixel data and labels.
        """
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)  
        self.dim = dim

    def __len__(self):
        """
        Return the total number of samples in the dataset.
        """
        return len(self.y)

    def __getitem__(self, idx):
        """
        Retrieve a single sample from the dataset.
        Args:
            idx (int): Index of the sample to retrieve.
        Returns:
            (torch.Tensor, torch.Tensor): Tuple of features and label.
        """
        X_sample = self.X[idx]
        y_sample = self.y[idx]

        if self.dim == "2d":
            X_sample = X_sample.unsqueeze(0)
            
        return X_sample, y_sample

In [ ]:
class EarlyStopper:
    def __init__(self, patience=3, min_delta=10):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_loss = float('inf')

    def early_stop(self, loss):
        if loss < self.min_loss:
            self.min_loss = loss
            self.counter = 0
        elif loss > (self.min_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False


In [ ]:
def create_train_test_dataloaders(X, y, test_size=0.2, batch_size=64, shuffle=True, dim="1d", max_samples_per_class=20000, bucket_size=10):
    """
    Split the DataFrame into train and test sets and create DataLoaders.
    Args:
        X (np.ndarray): Input features.
        y (np.ndarray): Target labels.
        test_size (float): Proportion of the data to be used as test data.
        batch_size (int): Number of samples per batch.
        shuffle (bool): Whether to shuffle the data.
        dim (str): Dimension type for the dataset.
        max_samples_per_class (int): Maximum samples to take for each unique y value.
    Returns:
        tuple: (train_loader, test_loader)
    """
    
    # Collect indices for each unique y value
    class_samples = defaultdict(list)
    
    for idx in range(len(y)):
        class_samples[y[idx]].append(idx)  # Store the index of the sample

    # Create a list to hold the limited indices
    limited_indices = []

    for class_label, indices in class_samples.items():
        limited_indices.extend(indices[:max_samples_per_class])  # Take the first 20,000 samples

    # Bucket y values
    bucketed_y = np.floor(y).astype(int)  
    bucketed_y = (y // 10) 

    # Create a subset of the dataset with the limited indices
    limited_X = X[limited_indices]
    limited_y = bucketed_y[limited_indices]

    # Shuffle the limited dataset if required
    if shuffle:
        shuffle_idxs = np.random.permutation(len(limited_y))
        limited_X = limited_X[shuffle_idxs]
        limited_y = limited_y[shuffle_idxs]

    # Split the limited dataset into train and test sets
    split_idx = int(len(limited_y) * (1 - test_size))
    X_train, X_test = limited_X[:split_idx], limited_X[split_idx:]
    y_train, y_test = limited_y[:split_idx], limited_y[split_idx:]

    # Create train and test datasets
    train_dataset = RULDataset(X_train, y_train, dim=dim)
    test_dataset = RULDataset(X_test, y_test, dim=dim)

    # Create DataLoaders for train and test sets
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [ ]:
def load_from_hdf5(filename="data.h5"):
    with h5py.File(filename, "r") as f:
        X = f["X"][:]
        y = f["y"][:]
    return X, y

In [ ]:
X, y = load_from_hdf5(filename="../sample_windows.h5")

In [ ]:
def evaluate_model(model, data_loader, criterion, device, print_loss=True):
    model.eval()
    overall_loss = 0.0

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Dynamically adjust input shape based on model type
            if isinstance(model, CNNRUL):
                inputs = inputs.permute(0, 2, 1)

            outputs = model(inputs).squeeze()
            targets = targets.long() 
            loss = criterion(outputs, targets)
            overall_loss += loss.item()

    avg_loss = overall_loss / len(data_loader)

    if print_loss:
        print(f"Test MSE: {avg_loss:.4f}")

    return avg_loss


def train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=5, track_test_loss=True):
    model.to(device)
    history = {'train_loss': [], 'test_loss': []}  
    early_stopper = EarlyStopper(patience=3, min_delta=.01)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            # Adjust shape based on model type
            if isinstance(model, CNNRUL):  # 1D CNN
                inputs = inputs.permute(0, 2, 1)
                
            optimizer.zero_grad()
            outputs = model(inputs).squeeze()
            targets = targets.long() 
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)

        if track_test_loss:
            avg_test_loss = evaluate_model(model, test_loader, criterion, device, print_loss=False)
            history['test_loss'].append(avg_test_loss)
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

            if early_stopper.early_stop(avg_test_loss):             
                break
        else:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}")

            if early_stopper.early_stop(avg_train_loss):             
                break

    return history

def plot_loss(history):
    epochs = range(1, len(history['train_loss']) + 1)

    plt.figure(figsize=(8, 5))
    plt.plot(epochs, history['train_loss'], label='Training Loss')
    plt.plot(epochs, history['test_loss'], label='Test Loss')
    
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training vs. Test Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

# 1d CNN
First, let's try a one dimensional CNN to capture temporal relationships within each feature.

In [ ]:
class CNNRUL(nn.Module):
    def __init__(self, num_features):
        super(CNNRUL, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * (50 // 2 // 2 // 2), 64)  # Adjust based on pooling
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

In [ ]:
unique_values, counts = np.unique(y, return_counts=True)

# Plot bar chart
plt.bar(unique_values, counts)

# Label axes
plt.xlabel("Y Value")
plt.ylabel("Count")
plt.title("Frequency of Y Values")

# Show plot
plt.show()

In [ ]:
counting = np.count_nonzero(y == 3)

counting

In [ ]:
train_loader, test_loader = create_train_test_dataloaders(X, y)



In [ ]:
print(type(train_loader))

In [ ]:
unique_labels = set()

# Assuming you have a DataLoader named `train_loader`
for inputs, targets in train_loader:
    unique_labels.update(targets.cpu().numpy())  # Use update to add multiple values

print(unique_labels)  # This will print unique labels

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [ ]:
num_feats = 44
model = CNNRUL(num_features=num_feats).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 10 

history = train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)

In [ ]:
def calculate_accuracy(outputs, targets):
    _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
    correct_predictions = (predicted == targets).sum().item()
    return correct_predictions

In [ ]:
model.eval()  # Set the model to evaluation mode
correct_predictions = 0
total_predictions = 0
predicted_values = []
correct_labels = []

with torch.no_grad():  # Disable gradient calculation
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # Move to the appropriate device
        
        # Adjust input shape if necessary
        if isinstance(model, CNNRUL):
            inputs = inputs.permute(0, 2, 1)

        outputs = model(inputs).squeeze()  # Get model predictions

        # Calculate accuracy using the provided function
        correct_predictions += calculate_accuracy(outputs, targets)
        total_predictions += targets.size(0)

        # Store predicted values and correct labels
        _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
        predicted_values.extend(predicted.cpu().numpy())
        correct_labels.extend(targets.cpu().numpy())

# Calculate overall accuracy
accuracy = correct_predictions / total_predictions * 100  # Convert to percentage

# Create confusion matrix
cm = confusion_matrix(correct_labels, predicted_values)

# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(correct_labels), yticklabels=np.unique(correct_labels))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Print overall accuracy
print(f"Overall Accuracy: {accuracy:.2f}%")

# 2d CNN
Now, let's try out a two dimensional CNN to capture interactions among features.

In [ ]:
class CNNRUL2D(nn.Module):
    def __init__(self, num_features):
        super(CNNRUL2D, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1)

        self.pool = nn.MaxPool2d(kernel_size=(2, 2)) 

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * (50 // 2 // 2 // 2) * (num_features // 2 // 2 // 2), 64)
        self.fc2 = nn.Linear(64, 10) 

    def forward(self, x):
        # Input shape: [batch, 1, 50, 44]
        x = torch.relu(self.conv1(x))
        x = self.pool(x)
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = torch.relu(self.conv3(x))
        x = self.pool(x)

        # Flatten and pass through FC layers
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

In [ ]:
train_loader, test_loader = create_train_test_dataloaders(X, y, dim="2d")

In [ ]:
num_feats = 44
model = CNNRUL2D(num_features=num_feats).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 10 

history = train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)
plot_loss(history)

# Hybrid CNN
Now, let's try a hybrid CNN that uses both 1d and 2d convolutions.

In [ ]:
class HybridCNN(nn.Module):
    def __init__(self, num_features, seq_length):
        super(HybridCNN, self).__init__()

        # 1d CNN Branch (Temporal Patterns)
        self.conv1d = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=3, padding=1)
        self.pool1d = nn.MaxPool1d(kernel_size=2)  

        # 2d CNN Branch (Feature Interactions)
        self.conv2d = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), padding=1)
        self.pool2d = nn.MaxPool2d(kernel_size=(2, 2))  

        # Compute Fully Connected Layer Sizes
        fc1d_input_size = 64 * (seq_length // 2)  # After 1D CNN
        fc2d_input_size = 32 * ((seq_length // 2) * (num_features // 2))  # After 2D CNN

        self.fc1 = nn.Linear(fc1d_input_size + fc2d_input_size, 128)  
        self.fc2 = nn.Linear(128, 10)  

    def forward(self, x):
        # 1d CNN Branch
        x1d = x.permute(0, 2, 1)  # Convert [batch, seq_length, features] → [batch, features, seq_length]
        x1d = self.conv1d(x1d) 
        x1d = self.pool1d(torch.relu(x1d)) 
        x1d = torch.flatten(x1d, start_dim=1)  

        # 2d CNN Branch
        x2d = x.unsqueeze(1)  # Convert [batch, seq_length, features] → [batch, 1, seq_length, features]
        x2d = self.conv2d(x2d)  
        x2d = self.pool2d(torch.relu(x2d))  
        x2d = torch.flatten(x2d, start_dim=1) 

        # Combine both branches
        x_combined = torch.cat((x1d, x2d), dim=1)

        x_combined = torch.relu(self.fc1(x_combined))
        return self.fc2(x_combined)

In [ ]:
train_loader, test_loader = create_train_test_dataloaders(X, y)

In [ ]:
num_feats = 44
seq_length = 50  
model = HybridCNN(num_features=num_feats, seq_length=seq_length).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 10

history = train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)
plot_loss(history)

In [ ]:
import matplotlib.pyplot as plt
import torch
import numpy as np

def plot_rul_predictions(model, test_loader, device):
    """
    Plots predicted RUL vs. actual RUL for the test set.

    Parameters:
        model (torch.nn.Module): Trained model for RUL prediction.
        test_loader (DataLoader): DataLoader containing test data.
        device (str): 'cuda' or 'cpu' based on availability.
    """
    model.eval()  # Set model to evaluation mode
    actual_rul = []
    predicted_rul = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            # Ensure the input shape is correct for different models
            if isinstance(model, TCN):  # TCN requires (batch, features, seq_len)
                inputs = inputs.permute(0, 2, 1)  
            
            outputs = model(inputs)  # Get predictions (now shape: [batch, 10])
            
            if isinstance(model.fc2, torch.nn.Linear) and model.fc2.out_features == 10:
                # take the most probable bucket
                predicted_rul.extend(torch.argmax(outputs, dim=1).cpu().numpy())

            actual_rul.extend(targets.cpu().numpy())

    # Convert to NumPy arrays for plotting
    actual_rul = np.array(actual_rul)
    predicted_rul = np.array(predicted_rul)

    # 🔹 Plot
    plt.figure(figsize=(8, 6))
    plt.scatter(actual_rul, predicted_rul, alpha=0.5, label="Predicted vs Actual", color="blue")
    plt.plot([min(actual_rul), max(actual_rul)], [min(actual_rul), max(actual_rul)], 'r--', label="Perfect Prediction")  
    plt.xlabel("Actual RUL")
    plt.ylabel("Predicted RUL")
    plt.title("Predicted vs. Actual RUL")
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
plot_rul_predictions(model, test_loader, device)

# More Complex Hybrid Model
The model's loss dropping drastically and then remaining stable might indicate underfitting. So, let's try adding additional convolutional layers.

In [ ]:
import torch
import torch.nn as nn

class HybridCNN(nn.Module):
    def __init__(self, num_features, seq_length):
        super(HybridCNN, self).__init__()

        # 1d CNN Branch (Temporal Feature Extraction)
        self.conv1d_1 = nn.Conv1d(in_channels=num_features, out_channels=32, kernel_size=3, padding=1)
        self.conv1d_2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv1d_3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.pool1d = nn.MaxPool1d(kernel_size=2)  
        self.bn1d_1 = nn.BatchNorm1d(32)
        self.bn1d_2 = nn.BatchNorm1d(64)
        self.bn1d_3 = nn.BatchNorm1d(128)

        ## 2d CNN Branch (Feature Interactions)
        self.conv2d_1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3, 3), padding=1)
        self.conv2d_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3, 3), padding=1)
        self.conv2d_3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding=1)
        self.pool2d = nn.MaxPool2d(kernel_size=(2, 2))  
        self.bn2d_1 = nn.BatchNorm2d(16)
        self.bn2d_2 = nn.BatchNorm2d(32)
        self.bn2d_3 = nn.BatchNorm2d(64)

        # Fully Connected Layers
        fc1d_input_size = 128 * (seq_length // 8)  # After three 1D CNN pooling layers
        fc2d_input_size = 64 * ((seq_length // 8) * (num_features // 8))  # After three 2D CNN pooling layers
        combined_fc_size = fc1d_input_size + fc2d_input_size

        self.fc1 = nn.Linear(combined_fc_size, 256)  
        self.fc2 = nn.Linear(256, 128)  
        self.fc3 = nn.Linear(128, 10)  

        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        # 1d CNN Branch
        x1d = x.permute(0, 2, 1)  # Convert [batch, seq_length, features] → [batch, features, seq_length]
        x1d = self.pool1d(torch.relu(self.bn1d_1(self.conv1d_1(x1d))))
        x1d = self.pool1d(torch.relu(self.bn1d_2(self.conv1d_2(x1d))))
        x1d = self.pool1d(torch.relu(self.bn1d_3(self.conv1d_3(x1d))))
        x1d = torch.flatten(x1d, start_dim=1)  

        # 2d CNN Branch
        x2d = x.unsqueeze(1)  # Convert [batch, seq_length, features] → [batch, 1, seq_length, features]
        x2d = self.pool2d(torch.relu(self.bn2d_1(self.conv2d_1(x2d))))
        x2d = self.pool2d(torch.relu(self.bn2d_2(self.conv2d_2(x2d))))
        x2d = self.pool2d(torch.relu(self.bn2d_3(self.conv2d_3(x2d))))
        x2d = torch.flatten(x2d, start_dim=1)  

        # Combine both branches
        x_combined = torch.cat((x1d, x2d), dim=1)

        # Fully Connected Layers
        x_combined = torch.relu(self.fc1(x_combined))
        x_combined = self.dropout(x_combined)
        x_combined = torch.relu(self.fc2(x_combined))
        x_combined = self.dropout(x_combined)

        return self.fc3(x_combined)  


In [ ]:
num_feats = 44
seq_length = 50  
model = HybridCNN(num_features=num_feats, seq_length=seq_length).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 10

history = train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)
plot_loss(history)

# Temporal CNN
Our more complex hybrid CNN was massively overfitting, so let's go in a different direction. Let's try a temporal CNN instead. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TemporalBlock(nn.Module):
    """
    A single block in a Temporal Convolutional Network (TCN).
    - Uses dilated causal convolutions, batch normalization, and residual connections.
    """
    def __init__(self, in_channels, out_channels, kernel_size, dilation, dropout=0.2):
        super(TemporalBlock, self).__init__()

        padding = (kernel_size - 1) * dilation // 2  # 🔹 Ensures output shape matches input

        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               padding=padding, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_channels)

        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               padding=padding, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.dropout = nn.Dropout(dropout)

        # 🔹 Adjust residual connection if the number of channels changes
        self.downsample = nn.Conv1d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else None

    def forward(self, x):
        res = x if self.downsample is None else self.downsample(x)  # 🔹 Match channel size if needed

        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout(x)

        return x + res  # 🔹 Ensures shape matches

class TCN(nn.Module):
    """
    Temporal Convolutional Network (TCN) for RUL Prediction.
    - Uses multiple TemporalBlocks with increasing dilation.
    - Outputs a single regression value per sequence.
    """
    def __init__(self, num_features, seq_length, num_channels=[64, 128, 256], kernel_size=3, dropout=0.2):
        super(TCN, self).__init__()

        layers = []
        num_levels = len(num_channels)

        for i in range(num_levels):
            dilation = 2 ** i  # 🔹 Exponentially increasing dilation rate
            in_channels = num_features if i == 0 else num_channels[i - 1]
            out_channels = num_channels[i]

            layers.append(TemporalBlock(in_channels, out_channels, kernel_size, dilation, dropout))

        self.network = nn.Sequential(*layers)
        self.fc = nn.Linear(num_channels[-1] * seq_length, 1)  # Fully connected output layer

    def forward(self, x):
        """
        x shape: [batch, seq_length, num_features]
        Expected shape: [batch, num_features, seq_length]
        """
        x = x.permute(0, 2, 1)  # 🔹 Convert [batch, seq_length, num_features] → [batch, num_features, seq_length]
        x = self.network(x)
        x = x.view(x.shape[0], -1)  # Flatten for FC layer
        return self.fc(x)


In [ ]:
num_feats = 44
seq_length = 50
model = TCN(num_features=num_feats, seq_length=seq_length).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
num_epochs = 20

history = train_model(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=num_epochs)
plot_loss(history)